In [0]:
%sql
create schema if not exists bike_data.silver

In [0]:
df=spark.table('bike_data.bronze.sales_details_crm')

In [0]:
df.display()

In [0]:
for c in df.columns:
    if c.startswith("sls_"):
        df = df.withColumnRenamed(c, c.replace("sls_", "", 1))


In [0]:
for c in df.columns:
    if c.startswith("cust"):
        df = df.withColumnRenamed(c, c.replace("cust", "customer", 1))

In [0]:
df.columns

### Checking Dupliactes

In [0]:
from pyspark.sql.functions import *

In [0]:
df.groupBy("ord_num", "prd_key") \
  .count() \
  .filter("count > 1") \
  .display()


In [0]:
df.describe()

In [0]:
df.display()

### Handling Null Values 

In [0]:
df.select(
    *[
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]
).display()

In [0]:
df = df.dropna(subset=["sales","price"])

In [0]:
df.count()

In [0]:
df.describe()

#### Changing datatype

In [0]:
df=df.withColumn("customer_id",col("customer_id").cast("int"))


In [0]:
df.display()

In [0]:
df.schema

In [0]:
# 10. Write Silver
df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("bike_data.silver.fact_sales_details")